In [ ]:
import pandas as pd
import numpy as np
import string
import json
import random

In [ ]:
## Static Config
locations = [
    {
        "column": "District",
        "name": "level1"
    },
    {
        "column": "Municipality/Union",
        "name": "level2"
    },
    {
        "column": "Ward Number",
        "name": "level3"
    }
]

latlong = ["latitude", "longitude"]
charts = [
    "water service level", 
    "sanitation service level", 
    "hygiene service level", 
    "health care waste service level",
    "environmental cleaning service level"
]
default = {
    "water service level" : ["Advanced", "Basic", "Limited", "No Service"],
    "sanitation service level": ["Advanced", "Basic", "Limited", "No Service"],
    "hygiene service level": ["Advanced", "Basic", "Limited", "No Service"],
    "health care waste service level": ["Advanced", "Basic", "Limited", "No Service"],
    "environmental cleaning service level": ["Advanced", "Basic", "Limited", "No Service"]
}

## Table config
table = [
    {
        "name": "water indicators",
        "column": "water service level",
        "indicators": [
            "Main Water Supply for the Facility",
            "Distance of Main Water Supply from Facility",
            "Water is available at the Time of Survey",
        ],
    },
    {
        "name": "sanitation indicators",
        "column": "sanitation service level",
        "indicators": [
            "Is there an improved toilet with permanent superstructure (Wall constructed of Bricks with concrete floor slab and Galvanised iron sheet roofing).",
            "Type of Toilet for Patients",
            "At least one toilet is usable",
            "Toilets dedicated for staff",
            "Toilets are in sex-separated or gender-neutral rooms",
            "Toilets have menstrual hygiene facilites",
            "Toilets accessible for people with limited mobility",
        ],
    },
    {
        "name": "hygiene indicators",
        "column": "hygiene service level",
        "indicators": [
            "Functional Hand Hygien Facility at Points of Care",
            "Functional handwashing facility at one or more toilets",
        ],
    },
    {
        "name": "health care waste indicators",
        "column": "health care waste service level",
        "indicators": [
            "Waste is Correctly Segregated into at Least Three Labelled Bins in the Consultation Area",
            "Main method for Disposing Infectious Waste",
            "Main method used for disposing Sharps Waste",
            
        ],
    },
    {
        "name": "environmental cleaning indicators",
        "column": "environmental cleaning service level",
        "indicators": [
            "Cleaning protocols are available",
            "All Staff Responsible for Cleaning Received Training",
            
        ],
    }
]

## Marker Detail
marker = {
    "name" : "Name_of_Health_facility",
    "color": [],
    "detail": []
}

main = {
    'name':'Health Facility',
    'column': 'Name_of_Health_facility',
    'indicators': [{
        'name': 'Water',
        'column': 'Water Service Level'
    },{
        'name': 'Sanitation',
        'column': 'Sanitation Service Level'
    },{
        'name': 'Hygiene',
        'column': 'Hygiene Service Level'
    },{
        'name': 'Health Care Waste',
        'column': 'Health Care Waste Service Level'
    },{
        'name': 'Environmental Cleaning',
        'column': 'Environmental Cleaning Service Level'
    }]
}


In [ ]:
df = pd.read_csv('bd_health_data.csv')

In [ ]:
## Rename level 2
def refineLevel2Name(x):
    if 'Municipality' in x:
        x = x.replace(' Municipality', '')
    if 'Municipality' in x or 'Union' in x:
        x = x.replace(' Union', '')
    return x

df['Municipality/Union'] = df['Municipality/Union'].apply(refineLevel2Name)

In [ ]:
## Filter Data by Municipality/Union - Agardari & Paurashava
# df = df[(df['Municipality/Union'] == 'Paurashava') | (df['Municipality/Union'] == 'Agardari')]
# df = df.reset_index(drop=True)

In [ ]:
## Delete Column with no name
df = df.loc[:, ~df.columns.str.contains('^Unnamed')]

In [ ]:
## Fill Column
#default_column = ["Identifier", "Woreda", "Kebele", "Name_of_Health_facility", "Latitude", "Longitude", "Type_of_healthy_facility"]
#for column in df.columns:
#    if column not in default_column and column != "Environmental Cleaning Service Level":
#        df[column] = df[column].apply(lambda x: random.choice(df[column].dropna().unique()))

In [ ]:
### Fill Empty Integer Values
numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
df_num = df.select_dtypes(include=numerics).drop(columns=['Latitude', 'Longitude'])
df_num = df_num.fillna(0.0).astype(np.int32)
df[list(df_num)] = df_num

In [ ]:
### Replace Empty String Values
df_str = df.select_dtypes(include=['object']).fillna("")
for strcol in list(df_str):
    df_str[strcol] = df_str[strcol].str.strip()
df[list(df_str)] = df_str

In [ ]:
## Generate Settings 
chars =list(string.ascii_uppercase)
chars_col = chars + [x+y for x in chars for y in chars]
keyname = lambda x,y: {a:y[b] if (a != 'data_point_id') else 'data_point_id' for b, a in enumerate(x)}

columns_length = len(list(df))
column_names = list(df)

In [ ]:
### Generate configs
index = chars_col[:columns_length]
configs = keyname(index, column_names)

marker_config = {}
chart_config = []
latlong_config = {}
for key in configs:
    if(configs[key].lower() in charts):
        chart_config.append({'key': key, 'name': configs[key], 'value': default[configs[key].lower()]})
    if(configs[key].lower() in latlong):
        latlong_config.update({configs[key].lower(): key})
    if(configs[key].lower() == marker['name'].lower()):
        marker_config = { "name" : key}

#### Locations
loc_config = {}
for loc in locations:
    column_index = ""
    for key in configs:
        if loc['column'].lower() == configs[key].lower():
            column_index = key
    loc_config.update({loc['name'].lower(): column_index})
    

#### Table
table_config = []
for tb in table:
    column_index = ""
    tb_indicators = []
    for key in configs:
        if tb['column'].lower() == configs[key].lower():
            column_index = key
        if configs[key].lower() in [x.lower() for x in tb['indicators']]:
            tb_indicators.append(key)
    table_config.append({
        "key": column_index,
        "name": tb['name'],
        "indicators": tb_indicators
    })

#### Marker
marker_color = [];
for mk in marker['color']:
    column = None
    andColumn = None
    for key in configs:
        if configs[key].lower() == mk['column'].lower():
            column = key
        if mk['and'] != None and configs[key].lower() == mk['and'].lower():
            andColumn = key
    marker_color.append({
        "name": mk['name'],
        "column": column,
        "action": mk['action'],
        "type": mk['type'],
        "value": mk['value'],
        "and": andColumn,
        "and_value": mk['and_value'],
        "color": mk['color']
    })
marker_config['color'] = marker_color

#### Marker Detail
marker_detail = [];
for mk in marker['detail']:
    column = None
    andColumn = None
    for key in configs:
        if configs[key].lower() == mk['column'].lower():
            column = key
        if mk['and'] != None and configs[key].lower() == mk['and'].lower():
            andColumn = key
    marker_detail.append({
        "name": mk['name'],
        "column": column,
        "action": mk['action'],
        "type": mk['type'],
        "value": mk['value'],
        "and": andColumn,
        "and_value": mk['and_value']
    })
marker_config['detail'] = marker_detail
    
## Main Table
main_config = {'name':main['name']}
indicators = []
for key in configs:
    if configs[key] == main['column']:
        main_config.update({'key':key})
    for i in main['indicators']:
        if configs[key] == i['column']:
            indicators.append({'name':i['name'],'key':key})
main_config.update({'indicators':indicators})
            
configs['charts'] = chart_config
configs['marker'] = marker_config 
configs['locations'] = loc_config
configs['latlong'] = latlong_config
configs['table'] = table_config
configs['main'] = main_config

In [ ]:
### Replace Dataset Columns
df.columns = index

In [ ]:
configs

In [ ]:
data = list(df.T.to_dict().values())
data[:2]

In [ ]:
### to json
df.to_json('../resources/js/data/bd_health_data.json', orient="records", indent=2)

with open('../resources/js/data/bd_health_config.json', 'w') as outfile:
    json.dump(configs, outfile)